In [1]:
%run initialising.ipynb

In [2]:
def formatP(p):
    if p >= 0.05:
        return str(round(p, 3))
    elif p >= 0.001:
        return str(round(p, 3)) + '*'
    else:
        return '<0.001*'

In [3]:
def independentTest(col1, col2):
    # no nan values in col1 or col2
    assert(pd.isnull(col1).values.any() == False)
    assert(pd.isnull(col2).values.any() == False)
    
    # n too small to test for normality    
    if len(col1) < 3 or len(col2) < 3:
        print('Mann-Whitney U test (n < 3)')
        stat, p = stats.mannwhitneyu(col1, col2)
        test = "Mann-Whitney U test"
    # data is non-normal
    elif stats.shapiro(col1)[1] < 0.05 or stats.shapiro(col2)[1] < 0.05:
        print('Mann-Whitney U test (p = %.4f, %.4f))' % (stats.shapiro(col1)[1], stats.shapiro(col2)[1]))
        stat, p = stats.mannwhitneyu(col1, col2)
        test = "Mann-Whitney U test"
    # data is normal
    else:
        if stats.bartlett(col1, col2)[1] < 0.05:
            print("Welch's t-test (unequal variance)")
            stat, p = stats.ttest_ind(col1, col2, equal_var=False, nan_policy='omit')
            test = 'Bartlett'
        else:
            print("Student's t-test (equal variance)")
            stat, p = stats.ttest_ind(col1, col2, equal_var=True, nan_policy='omit')
            test = "Student's t-test"
    return p, test

def pairedTest(col1, col2):
    # since Wilcoxon test does not have nan_policy parameter,
    # must manually drop nan using this method
    # note that if nan is dropped from 1 column, it must be
    # dropped from the other
    colDf = pd.concat([col1, col2], axis=1).dropna()
    col1 = colDf.iloc[:,0]
    col2 = colDf.iloc[:,1]
    
    if len(col1) < 3 or len(col2) < 3:
        print('Wilcoxon signed-rank test (n < 3)')
        stat, p = stats.mannwhitneyu(col1, col2)
        test = 'Wilcoxon signed-rank test'
    elif stats.shapiro(col1)[1] < 0.05 or stats.shapiro(col2)[1] < 0.05:
        print('Wilcoxon signed-rank test (p = %.4f, %.4f))' % (stats.shapiro(col1)[1], stats.shapiro(col2)[1]))
        stat, p = stats.wilcoxon(col1, col2)
        test = 'Wilcoxon signed-rank test'
    else: # data is normally distributed
        print('Paired t-tests')
        stat, p = stats.ttest_rel(col1, col2, nan_policy='omit')
        test = 'Paired t-test'
    return p, test

In [4]:
def compare_ex30_vs_ld30(measure_type, threshold):
    '''Prints statistical analysis comparing %acc of EX30 vs LD30 for a
    given measure_type and minimum expansion threshold'''
    
    if threshold:
        print("Is the %%accuracy of EX30 significantly different to LD30, when requested expansion > %s mm?\n" % threshold)
    else:
        print("Is the %accuracy of EX30 significantly different to LD30, for any level of requested expansion?")

    all_ex30 = pd.Series([])
    all_ld30 = pd.Series([])
    
    newDf = pd.DataFrame(columns=['Tooth Type', 'LD30 %Accuracy', 'SD1', 'n1', 'EX30 %Accuracy', 'SD2', 'n2', 'p', 'test'])
    
    for i, j in zip([1, 3], [2, 4]):
        for toothNum in range(3, 8):
            if measure_type == '.cc' and toothNum == 7:
                continue

            antimere = '%s%s-%s%s' % (i, toothNum, j, toothNum)
            full_antimere = '%s%s-%s%s%s' % (i, toothNum, j, toothNum, measure_type)

            col1 = getColumn(df, '%s %%acc' % full_antimere, antimere, threshold, measure_type, 'EX30').dropna()
            col2 = getColumn(df, '%s %%acc' % full_antimere, antimere, threshold, measure_type, 'LD30').dropna()
            
            print('---%s---' % full_antimere)

            p, test = independentTest(col1, col2)
            
            print('EX30: mean=%s\tstd=%s\tn=%s' % (col1.mean(), col1.std(), col1.count()))
            print('LD30: mean=%s\tstd=%s\tn=%s' % (col2.mean(), col2.std(), col2.count()))
            s = 'p=%s' % p
            if p < 0.05:
                s = '*' + s
            print('%s\n' % s)
            
            all_ex30 = all_ex30.append(col1)
            all_ld30 = all_ld30.append(col2)
            
            # add row to table (newDf)
            k = 0 if pd.isnull(newDf.index.max()) else newDf.index.max()+1       
            newDf.loc[k] = [antimere_dict[antimere], col2.mean(), col2.std(), col2.count(), col1.mean(), col1.std(), col1.count(), formatP(p), test]
    
    # calculate 'overall' statistic
    p, test = independentTest(all_ex30, all_ld30)
    print('ALL EX30%s: mean=%s\tstd=%s\tn=%s' % (measure_type, all_ex30.mean(), all_ex30.std(), all_ex30.count()))
    print('ALL LD30%s: mean=%s\tstd=%s\tn=%s' % (measure_type, all_ld30.mean(), all_ld30.std(), all_ld30.count()))
    
    s = 'p=%s' % p
    if p < 0.05:
        s = '*' + s
    print('%s\n\n\n' % s)
    
    # add 'overall' statistic to df
    k = 0 if pd.isnull(newDf.index.max()) else newDf.index.max()+1       
    newDf.loc[k] = ['Total', all_ld30.mean(), all_ld30.std(), all_ld30.count(), all_ex30.mean(), all_ex30.std(), all_ex30.count(), formatP(p), test]
    newDf = newDf.round({'LD30 %Accuracy': 1,
                 'SD1': 1,
                 'n1': 0,
                 'EX30 %Accuracy': 1,
                 'SD2': 1,
                 'n2': 0
    })
    newDf.to_csv(os.path.join(OUTPUT_DIR, 'tables/ex30-vs-ld30-%s-%s.csv'%(measure_type, threshold)), index=False)
    
    return

In [5]:
def compare_predicted_vs_achieved(material, measure_type, threshold):
    '''Prints statistical analysis comparing %acc of  vs LD30 for a
    given measure_type and minimum expansion threshold'''
    
    assert(material in ['LD30', 'EX30'])
    
    if threshold:
        print("Is the actual expansion of %s significantly different to its predicted expansion, for cases where predicted expansion >%s mm?\n"\
          % (material, threshold))
    else:
        print("Is the actual expansion of %s significantly different to its predicted expansion, for any level of predicted expansion?\n"\
          % (material))
    
    # create new df which will serve as table of results
    newDf = pd.DataFrame(columns=['Tooth Type', 'n', 'Predicted Expansion', 'SD1', 'Achieved Expansion', 'SD2', 'p', '%Accuracy', 'SD3', 'test'])
    
    for i, j in zip([1, 3], [2, 4]):
        for toothNum in range(3, 8):
            if measure_type == '.cc' and toothNum == 7:
                continue

            antimere = '%s%s-%s%s' % (i, toothNum, j, toothNum)
            full_antimere = '%s%s-%s%s%s' % (i, toothNum, j, toothNum, measure_type)


            col1 = getColumn(df, 'aΔ%s%s' % (antimere, measure_type), antimere, threshold, measure_type, material)
            col2 = getColumn(df, 'pΔ%s%s' % (antimere, measure_type), antimere, threshold, measure_type, material)

            print('---%s---' % full_antimere)
            p, test = pairedTest(col1, col2)

            print('FΔ: mean=%s\tstd=%s\tn=%s' % (col1.mean(), col1.std(), col1.count()))
            print('PΔ: mean=%s\tstd=%s\tn=%s' % (col2.mean(), col2.std(), col2.count()))
            
            s = 'p=%s' % p
            if p < 0.05:
                s = '*' + s
            print('%s\n' % (s))
            
            acc = getColumn(df, '%s %%acc' % full_antimere, antimere, threshold, measure_type, 'LD30')
            
            ## add row to table (newDf)
            k = 0 if pd.isnull(newDf.index.max()) else newDf.index.max()+1
            n = min([col1.count(), col2.count()])
            newDf.loc[k] = [antimere_dict[antimere], n, col2.mean(), col2.std(), col1.mean(), col1.std(), formatP(p), acc.mean(), acc.std(), test]
    newDf = newDf.round({'n':0,
                 'Predicted Expansion':2,
                 'SD1':2,
                 'Achieved Expansion': 2,
                 'SD2': 2,
                 '%Accuracy':1,
                 'SD3':1
    })
    newDf.to_csv(os.path.join(OUTPUT_DIR, 'tables/%s-%s-%s-acc.csv'%(material, measure_type, threshold)), index=False)
    return

In [6]:
for measure_type in MEASURE_TYPES:
    compare_ex30_vs_ld30(measure_type, THRESHOLD)

Is the %accuracy of EX30 significantly different to LD30, when requested expansion > 3.0 mm?

---13-23.cc---
Student's t-test (equal variance)
EX30: mean=80.29272308125115	std=13.86859682501173	n=13
LD30: mean=77.3882709763687	std=16.246508317890118	n=19
p=0.60272477183

---14-24.cc---
Mann-Whitney U test (p = 0.5984, 0.0499))
EX30: mean=77.23141399154606	std=22.05422076004985	n=25
LD30: mean=83.76207058520345	std=16.333874237836014	n=44
p=0.0877865061948

---15-25.cc---
Student's t-test (equal variance)
EX30: mean=68.30061383666492	std=18.520576806241156	n=23
LD30: mean=82.61598159560957	std=18.1897000637605	n=35
*p=0.00516165317857

---16-26.cc---
Student's t-test (equal variance)
EX30: mean=49.089240381669335	std=21.103237086421377	n=5
LD30: mean=71.73778012736454	std=24.97515081524634	n=16
p=0.0836395154589

---33-43.cc---
Student's t-test (equal variance)
EX30: mean=85.73497122862864	std=17.785513672553236	n=10
LD30: mean=86.4107139346012	std=17.639135017925796	n=25
p=0.9192526705

In [7]:
for measure_type in MEASURE_TYPES:
    compare_predicted_vs_achieved('LD30', measure_type, THRESHOLD)

Is the actual expansion of LD30 significantly different to its predicted expansion, for cases where predicted expansion >3.0 mm?

---13-23.cc---
Paired t-tests
FΔ: mean=3.378947368421052	std=1.0881068918939603	n=19
PΔ: mean=4.321052631578947	std=0.9015905373704975	n=19
*p=3.18603886854e-06

---14-24.cc---
Wilcoxon signed-rank test (p = 0.0094, 0.0000))
FΔ: mean=3.7090909090909094	std=1.1978310207763185	n=44
PΔ: mean=4.420454545454545	std=1.1294327749957809	n=44
*p=6.41751239017e-07

---15-25.cc---
Wilcoxon signed-rank test (p = 0.1574, 0.0011))
FΔ: mean=3.5200000000000005	std=1.0747913614224092	n=35
PΔ: mean=4.300000000000001	std=1.0737509843863187	n=35
*p=2.73853264388e-05

---16-26.cc---
Paired t-tests
FΔ: mean=2.5687499999999996	std=0.8889835019091561	n=16
PΔ: mean=3.5999999999999988	std=0.38815804341359067	n=16
*p=0.000449470929011

---33-43.cc---
Wilcoxon signed-rank test (p = 0.0707, 0.0021))
FΔ: mean=3.403999999999999	std=1.0022807324630492	n=25
PΔ: mean=3.96	std=0.9133272505880

/Users/Richard/anaconda/envs/py36/lib/python3.6/site-packages/scipy/stats/morestats.py:2397: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
